# 10. Image Quality Assessment (v2) 

The objective is to quantify the performance of image filters for three period: 
1. Aug to Oct - Beginning of season
2. Nov to Jan - Middle of season
3. Feb to Apr - End of season

In [1]:
beginning = [
    'sentinel-1_image_VH_2018-08-12.png',
    'sentinel-1_image_VH_2018-09-05.png',
    'sentinel-1_image_VH_2018-10-11.png'
]

middle = [
    'sentinel-1_image_VH_2018-11-04.png',
    'sentinel-1_image_VH_2018-12-10.png',
    'sentinel-1_image_VH_2019-01-03.png'
]

end = [
    'sentinel-1_image_VH_2019-02-08.png',
    'sentinel-1_image_VH_2019-03-04.png',
    'sentinel-1_image_VH_2019-04-09.png'
]

**Quantitative Metrics:**
1. Speckle Suppression Index:	$SSI = \frac{s_F}{\mu_F} \cdot \frac{\mu_M}{s_M}$ [1]
2. Speckle Suppresion and Mean Preservation Index: $SMPI = (1 + |\mu_M - \mu_F|) \cdot \frac{\sigma_F}{\sigma_M}$	[1]
3. Mean Preservation Index: $MPI = |\frac{\mu_M - \mu_F}{\mu_M}|$ [1]
4. Mean Preservation Speckle Suppression Index: $MPSSI = |1 - \frac{\mu_F}{\mu_M}| \cdot \frac{\sigma_F}{\sigma_M}=MPI \cdot \frac{\sigma_F}{\sigma_M}$ [1]
5. Equivalent Number of Looks: $ENL=\frac{\mu_F^2}{\sigma_F^2}$ [1]
6. Edge Save Index Horizontal/Vertical [2, 3]. Let $P$ be the maximum row number and $Q$ be the maximum column.
- $ESIH=\frac{\sum_{m=1}^{P} \sum_{n=1}^{Q-1} |F_{m,n+1} - F{m,n}|}{\sum_{m=1}^{P}\sum_{n=1}^{Q-1} |M_{m,n+1} - M{m,n}|}$ 
- $ESIV=\frac{\sum_{n=1}^{Q} \sum_{m=1}^{P-1} |F_{m+1,n} - F{m,n}|}{\sum_{n=1}^{Q}\sum_{m=1}^{P-1} |M_{m+1,n} - M{m,n}|}$ 

In [2]:
import pandas as pd
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def ssi(original, filtered): 
    # Speckle Suppression Index (SSI) (smaller, better)
    return (np.std(filtered) / np.mean(filtered)) * (np.mean(original) / np.std(original))

def smpi(original, filtered):
    # Speckle Suppression and Mean Preservation Index (SMPI) (smaller, better)
    Q = 1 + np.abs(np.mean(original) - np.mean(filtered))
    return Q * np.std(filtered) / np.std(original)

def mpi(original, filtered):
    # Mean Preservation Index (MPI), (smaller, better)
    return np.abs((np.mean(original) - np.mean(filtered)) / np.mean(original))
    
def mpssi(original, filtered):
    # Mean Preservation Speckle Suppression Index (MPSSI), (smaller, better)
    return mpi(original, filtered) * (np.std(filtered) / np.std(original))
    
def enl(image):
    # Equivalent Number of Looks (ENL) (higher, better)
    return (np.mean(image) ** 2) / np.var(image)

def esih(original, filtered):
    # Efficient Implementation of Edge Save Index Horizontal (ESIH)
    rows, cols = original.shape
    numerator = np.sum(np.abs(filtered[:, 1:cols] - filtered[:, :cols-1]).flatten())
    denominator = np.sum(np.abs(original[:, 1:cols] - original[:, :cols-1]).flatten())
    return numerator / denominator

def esiv(original, filtered):
    # Efficient Implementation of Edge Save Index Vertical (ESIH)
    rows, cols = original.shape 
    numerator = np.sum(np.abs(filtered[1:rows, :] - filtered[:rows-1, :]).flatten())
    denominator = np.sum(np.abs(original[1:rows, :] - original[:rows-1, :]).flatten())
    return numerator / denominator

## Classical Filters

In [5]:
directories = {
    'Unfiltered': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/selected_images',
    'Mean Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/boxcar',
    'Median Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/median',
    'Lee Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/lee',
    'Frost Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/frost',
    'Gamma MAP Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/gamma_map',
}

metrics = {
    'SSI': ssi, 
    'SMPI': smpi, 
    'MPI': mpi,
    'MPSSI': mpssi,
    'ENL': enl,
    'ESIH': esih,
    'ESIV': esiv
}

### Beginning of Season

In [8]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in beginning:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            # print(results)
            median = np.median(results)
            output.loc[directory, metric] = median


Mean Filter SSI sentinel-1_image_VH_2018-08-12.png
Mean Filter SSI sentinel-1_image_VH_2018-09-05.png
Mean Filter SSI sentinel-1_image_VH_2018-10-11.png
Mean Filter SMPI sentinel-1_image_VH_2018-08-12.png
Mean Filter SMPI sentinel-1_image_VH_2018-09-05.png
Mean Filter SMPI sentinel-1_image_VH_2018-10-11.png
Mean Filter MPI sentinel-1_image_VH_2018-08-12.png
Mean Filter MPI sentinel-1_image_VH_2018-09-05.png
Mean Filter MPI sentinel-1_image_VH_2018-10-11.png
Mean Filter MPSSI sentinel-1_image_VH_2018-08-12.png
Mean Filter MPSSI sentinel-1_image_VH_2018-09-05.png
Mean Filter MPSSI sentinel-1_image_VH_2018-10-11.png
Mean Filter ENL sentinel-1_image_VH_2018-08-12.png
Mean Filter ENL sentinel-1_image_VH_2018-09-05.png
Mean Filter ENL sentinel-1_image_VH_2018-10-11.png
Mean Filter ESIH sentinel-1_image_VH_2018-08-12.png
Mean Filter ESIH sentinel-1_image_VH_2018-09-05.png
Mean Filter ESIH sentinel-1_image_VH_2018-10-11.png
Mean Filter ESIV sentinel-1_image_VH_2018-08-12.png
Mean Filter ESIV s

In [9]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
Mean Filter,0.873821,1.00611,0.0010625,0.000928396,66.3387,0.345733,0.337215
Median Filter,0.829458,1.56737,0.0058632,0.00489179,73.7745,0.344666,0.336291
Lee Filter,0.892776,0.908555,0.000118751,0.000106006,63.681,0.424712,0.413614
Frost Filter,0.890391,1.29736,0.00322486,0.00283739,64.0226,0.422061,0.410872
Gamma MAP Filter,0.929716,29.0782,0.295884,0.193847,57.964,0.461744,0.467833


In [10]:
output.to_excel('../results/image_quality/beginning/classic_filters.xlsx')

### Middle of Season

In [11]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in middle:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            # print(results)
            median = np.median(results)
            output.loc[directory, metric] = median


Mean Filter SSI sentinel-1_image_VH_2018-11-04.png
Mean Filter SSI sentinel-1_image_VH_2018-12-10.png
Mean Filter SSI sentinel-1_image_VH_2019-01-03.png
Mean Filter SMPI sentinel-1_image_VH_2018-11-04.png
Mean Filter SMPI sentinel-1_image_VH_2018-12-10.png
Mean Filter SMPI sentinel-1_image_VH_2019-01-03.png
Mean Filter MPI sentinel-1_image_VH_2018-11-04.png
Mean Filter MPI sentinel-1_image_VH_2018-12-10.png
Mean Filter MPI sentinel-1_image_VH_2019-01-03.png
Mean Filter MPSSI sentinel-1_image_VH_2018-11-04.png
Mean Filter MPSSI sentinel-1_image_VH_2018-12-10.png
Mean Filter MPSSI sentinel-1_image_VH_2019-01-03.png
Mean Filter ENL sentinel-1_image_VH_2018-11-04.png
Mean Filter ENL sentinel-1_image_VH_2018-12-10.png
Mean Filter ENL sentinel-1_image_VH_2019-01-03.png
Mean Filter ESIH sentinel-1_image_VH_2018-11-04.png
Mean Filter ESIH sentinel-1_image_VH_2018-12-10.png
Mean Filter ESIH sentinel-1_image_VH_2019-01-03.png
Mean Filter ESIV sentinel-1_image_VH_2018-11-04.png
Mean Filter ESIV s

In [12]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
Mean Filter,0.80475,0.920858,0.00105872,0.000851102,129.091,0.307584,0.30312
Median Filter,0.78775,1.21173,0.0037399,0.00311364,136.044,0.318476,0.312446
Lee Filter,0.822673,0.824064,1.03651e-05,9.13883e-06,122.801,0.387289,0.379923
Frost Filter,0.825561,1.22625,0.00352617,0.00288377,122.334,0.38797,0.380571
Gamma MAP Filter,0.89124,26.122,0.295478,0.185517,105.473,0.439178,0.446337


In [13]:
output.to_excel('../results/image_quality/middle/classic_filters.xlsx')

### End of Season

In [14]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in end:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            # print(results)
            median = np.median(results)
            output.loc[directory, metric] = median


Mean Filter SSI sentinel-1_image_VH_2019-02-08.png
Mean Filter SSI sentinel-1_image_VH_2019-03-04.png
Mean Filter SSI sentinel-1_image_VH_2019-04-09.png
Mean Filter SMPI sentinel-1_image_VH_2019-02-08.png
Mean Filter SMPI sentinel-1_image_VH_2019-03-04.png
Mean Filter SMPI sentinel-1_image_VH_2019-04-09.png
Mean Filter MPI sentinel-1_image_VH_2019-02-08.png
Mean Filter MPI sentinel-1_image_VH_2019-03-04.png
Mean Filter MPI sentinel-1_image_VH_2019-04-09.png
Mean Filter MPSSI sentinel-1_image_VH_2019-02-08.png
Mean Filter MPSSI sentinel-1_image_VH_2019-03-04.png
Mean Filter MPSSI sentinel-1_image_VH_2019-04-09.png
Mean Filter ENL sentinel-1_image_VH_2019-02-08.png
Mean Filter ENL sentinel-1_image_VH_2019-03-04.png
Mean Filter ENL sentinel-1_image_VH_2019-04-09.png
Mean Filter ESIH sentinel-1_image_VH_2019-02-08.png
Mean Filter ESIH sentinel-1_image_VH_2019-03-04.png
Mean Filter ESIH sentinel-1_image_VH_2019-04-09.png
Mean Filter ESIV sentinel-1_image_VH_2019-02-08.png
Mean Filter ESIV s

In [15]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
Mean Filter,0.776786,0.897434,0.00104505,0.000810389,234.722,0.308995,0.30327
Median Filter,0.751701,1.06561,0.00298775,0.00211384,250.649,0.320473,0.312634
Lee Filter,0.787899,0.788326,3.59544e-06,2.83286e-06,228.147,0.387509,0.378845
Frost Filter,0.790992,1.1785,0.00341464,0.00260291,226.367,0.388393,0.379792
Gamma MAP Filter,0.886782,27.2597,0.295141,0.184458,180.104,0.43928,0.447015


In [16]:
output.to_excel('../results/image_quality/end/classic_filters.xlsx')

## BM3D

In [18]:
directories = {f'BM3D_sigma_psd_{i}': f'../results/bm3d_images/sigma_psd_{i}' for i in [10, 20, 30, 40, 50]}
directories['Unfiltered'] = '../results/selected_images'
directories

{'BM3D_sigma_psd_10': '../results/bm3d_images/sigma_psd_10',
 'BM3D_sigma_psd_20': '../results/bm3d_images/sigma_psd_20',
 'BM3D_sigma_psd_30': '../results/bm3d_images/sigma_psd_30',
 'BM3D_sigma_psd_40': '../results/bm3d_images/sigma_psd_40',
 'BM3D_sigma_psd_50': '../results/bm3d_images/sigma_psd_50',
 'Unfiltered': '../results/selected_images'}

### Beginning of Season

In [19]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in beginning:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 ESIH sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 ESIH sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 ES

In [20]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
BM3D_sigma_psd_10,0.940907,0.943863,2.08282e-05,1.95978e-05,57.3326,0.559174,0.54269
BM3D_sigma_psd_20,0.889093,0.902214,9.78142e-05,8.69745e-05,64.2096,0.330432,0.308867
BM3D_sigma_psd_30,0.85912,0.880961,0.000168496,0.000144783,68.7682,0.245818,0.226806
BM3D_sigma_psd_40,0.839952,0.856125,0.000127622,0.00010721,71.9425,0.203168,0.186573
BM3D_sigma_psd_50,0.827391,0.828109,8.86749e-06,7.3791e-06,74.1435,0.179377,0.164629


In [21]:
output.to_excel('../results/image_quality/beginning/bm3d_filter.xlsx')

### Middle of Season

In [22]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in middle:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 ESIH sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 ESIH sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 ES

In [23]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
BM3D_sigma_psd_10,0.879903,0.886907,5.69833e-05,4.99756e-05,107.014,0.49576,0.479613
BM3D_sigma_psd_20,0.779564,0.796845,0.000143881,0.000112055,135.589,0.240964,0.220644
BM3D_sigma_psd_30,0.738726,0.739761,2.66901e-05,1.96717e-05,151.693,0.162213,0.146517
BM3D_sigma_psd_40,0.71879,0.729919,0.00010758,7.69726e-05,160.927,0.133886,0.121498
BM3D_sigma_psd_50,0.705872,0.730635,0.000254549,0.000178547,167.395,0.118742,0.108342


In [24]:
output.to_excel('../results/image_quality/middle/bm3d_filter.xlsx')

### End of Season

In [25]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in end:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 MPI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 MPSSI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 ENL sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 ESIH sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 ESIH sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 ES

In [26]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
BM3D_sigma_psd_10,0.841929,0.844688,2.17171e-05,1.82846e-05,199.805,0.464512,0.447466
BM3D_sigma_psd_20,0.725279,0.73072,6.1423e-05,4.1247e-05,269.244,0.213673,0.188839
BM3D_sigma_psd_30,0.679764,0.680834,1.05721e-05,7.18647e-06,306.507,0.15052,0.131915
BM3D_sigma_psd_40,0.65594,0.665709,0.000102517,7.17248e-05,329.176,0.126976,0.111591
BM3D_sigma_psd_50,0.639848,0.659076,0.000225805,0.000143017,345.942,0.114036,0.10056


In [27]:
output.to_excel('../results/image_quality/end/bm3d_filter.xlsx')

## Boxcar + DFT Edge

In [31]:
directories = {f'radius_{i}': f'../results/boxcar_edge/radius_{i}' for i in range(4,7)}
directories['Unfiltered'] = '../results/selected_images'

### Beginning of Season

In [32]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in beginning:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


radius_4 SSI sentinel-1_image_VH_2018-08-12.png
radius_4 SSI sentinel-1_image_VH_2018-09-05.png
radius_4 SSI sentinel-1_image_VH_2018-10-11.png
radius_4 SMPI sentinel-1_image_VH_2018-08-12.png
radius_4 SMPI sentinel-1_image_VH_2018-09-05.png
radius_4 SMPI sentinel-1_image_VH_2018-10-11.png
radius_4 MPI sentinel-1_image_VH_2018-08-12.png
radius_4 MPI sentinel-1_image_VH_2018-09-05.png
radius_4 MPI sentinel-1_image_VH_2018-10-11.png
radius_4 MPSSI sentinel-1_image_VH_2018-08-12.png
radius_4 MPSSI sentinel-1_image_VH_2018-09-05.png
radius_4 MPSSI sentinel-1_image_VH_2018-10-11.png
radius_4 ENL sentinel-1_image_VH_2018-08-12.png
radius_4 ENL sentinel-1_image_VH_2018-09-05.png
radius_4 ENL sentinel-1_image_VH_2018-10-11.png
radius_4 ESIH sentinel-1_image_VH_2018-08-12.png
radius_4 ESIH sentinel-1_image_VH_2018-09-05.png
radius_4 ESIH sentinel-1_image_VH_2018-10-11.png
radius_4 ESIV sentinel-1_image_VH_2018-08-12.png
radius_4 ESIV sentinel-1_image_VH_2018-09-05.png
radius_4 ESIV sentinel-1_i

In [33]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
radius_4,3.09613,70.0475,0.174137,0.475576,5.29487,2.73574,2.74361
radius_5,3.12816,70.7473,0.179979,0.480431,5.11709,2.76931,2.77801
radius_6,3.10044,70.2382,0.181362,0.476847,5.06352,2.7565,2.76124


In [34]:
output.to_excel('../results/image_quality/beginning/boxcar_dft_edge_filter.xlsx')

### Middle of Season

In [35]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in middle:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


radius_4 SSI sentinel-1_image_VH_2018-11-04.png
radius_4 SSI sentinel-1_image_VH_2018-12-10.png
radius_4 SSI sentinel-1_image_VH_2019-01-03.png
radius_4 SMPI sentinel-1_image_VH_2018-11-04.png
radius_4 SMPI sentinel-1_image_VH_2018-12-10.png
radius_4 SMPI sentinel-1_image_VH_2019-01-03.png
radius_4 MPI sentinel-1_image_VH_2018-11-04.png
radius_4 MPI sentinel-1_image_VH_2018-12-10.png
radius_4 MPI sentinel-1_image_VH_2019-01-03.png
radius_4 MPSSI sentinel-1_image_VH_2018-11-04.png
radius_4 MPSSI sentinel-1_image_VH_2018-12-10.png
radius_4 MPSSI sentinel-1_image_VH_2019-01-03.png
radius_4 ENL sentinel-1_image_VH_2018-11-04.png
radius_4 ENL sentinel-1_image_VH_2018-12-10.png
radius_4 ENL sentinel-1_image_VH_2019-01-03.png
radius_4 ESIH sentinel-1_image_VH_2018-11-04.png
radius_4 ESIH sentinel-1_image_VH_2018-12-10.png
radius_4 ESIH sentinel-1_image_VH_2019-01-03.png
radius_4 ESIV sentinel-1_image_VH_2018-11-04.png
radius_4 ESIV sentinel-1_image_VH_2018-12-10.png
radius_4 ESIV sentinel-1_i

In [36]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
radius_4,4.97053,133.305,0.248273,0.943081,3.5177,4.34516,4.35054
radius_5,5.09075,140.038,0.254712,0.949179,3.46926,4.4824,4.48803
radius_6,5.10032,144.595,0.252471,0.951217,3.45625,4.55381,4.55376


In [37]:
output.to_excel('../results/image_quality/middle/boxcar_dft_edge_filter.xlsx')

### End of Season

In [38]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in end:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


radius_4 SSI sentinel-1_image_VH_2019-02-08.png
radius_4 SSI sentinel-1_image_VH_2019-03-04.png
radius_4 SSI sentinel-1_image_VH_2019-04-09.png
radius_4 SMPI sentinel-1_image_VH_2019-02-08.png
radius_4 SMPI sentinel-1_image_VH_2019-03-04.png
radius_4 SMPI sentinel-1_image_VH_2019-04-09.png
radius_4 MPI sentinel-1_image_VH_2019-02-08.png
radius_4 MPI sentinel-1_image_VH_2019-03-04.png
radius_4 MPI sentinel-1_image_VH_2019-04-09.png
radius_4 MPSSI sentinel-1_image_VH_2019-02-08.png
radius_4 MPSSI sentinel-1_image_VH_2019-03-04.png
radius_4 MPSSI sentinel-1_image_VH_2019-04-09.png
radius_4 ENL sentinel-1_image_VH_2019-02-08.png
radius_4 ENL sentinel-1_image_VH_2019-03-04.png
radius_4 ENL sentinel-1_image_VH_2019-04-09.png
radius_4 ESIH sentinel-1_image_VH_2019-02-08.png
radius_4 ESIH sentinel-1_image_VH_2019-03-04.png
radius_4 ESIH sentinel-1_image_VH_2019-04-09.png
radius_4 ESIV sentinel-1_image_VH_2019-02-08.png
radius_4 ESIV sentinel-1_image_VH_2019-03-04.png
radius_4 ESIV sentinel-1_i

In [40]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
radius_4,6.53307,180.572,0.265756,1.21831,3.26298,5.25475,5.25651
radius_5,6.43876,175.391,0.262856,1.18262,3.30737,5.17458,5.17439
radius_6,6.47301,177.229,0.273263,1.19528,3.13286,5.20703,5.21005


In [41]:
output.to_excel('../results/image_quality/end/boxcar_dft_edge_filter.xlsx')

## BM3D + DFT Edge

In [42]:
directories = {f'radius_{i}': f'../results/bm3d_edge/radius_{i}' for i in range(4,7)}
directories['Unfiltered'] = '../results/selected_images'
directories

{'radius_4': '../results/bm3d_edge/radius_4',
 'radius_5': '../results/bm3d_edge/radius_5',
 'radius_6': '../results/bm3d_edge/radius_6',
 'Unfiltered': '../results/selected_images'}

In [43]:
seasons = {
    'beginning': beginning, 
    'middle': middle, 
    'end': end
}

for season in seasons:
    output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
    for directory in directories:
        if directory != 'Unfiltered':        
            for metric in metrics:
                results = []
                for file in seasons[season]:
                    print(directory, metric, file)
                    if metric == 'ENL':
                        image = os.path.join(directories[directory], file)
                        image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                        results.append(metrics[metric](image))
                    else:
                        original = os.path.join(directories['Unfiltered'], file)
                        original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                        filtered = os.path.join(directories[directory], file)
                        filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                        results.append(metrics[metric](original, filtered))
                median = np.median(results)
                output.loc[directory, metric] = median
    output.to_excel(f'../results/image_quality/{season}/bm3d_dft_edge_filter.xlsx')

radius_4 SSI sentinel-1_image_VH_2018-08-12.png
radius_4 SSI sentinel-1_image_VH_2018-09-05.png
radius_4 SSI sentinel-1_image_VH_2018-10-11.png
radius_4 SMPI sentinel-1_image_VH_2018-08-12.png
radius_4 SMPI sentinel-1_image_VH_2018-09-05.png
radius_4 SMPI sentinel-1_image_VH_2018-10-11.png
radius_4 MPI sentinel-1_image_VH_2018-08-12.png
radius_4 MPI sentinel-1_image_VH_2018-09-05.png
radius_4 MPI sentinel-1_image_VH_2018-10-11.png
radius_4 MPSSI sentinel-1_image_VH_2018-08-12.png
radius_4 MPSSI sentinel-1_image_VH_2018-09-05.png
radius_4 MPSSI sentinel-1_image_VH_2018-10-11.png
radius_4 ENL sentinel-1_image_VH_2018-08-12.png
radius_4 ENL sentinel-1_image_VH_2018-09-05.png
radius_4 ENL sentinel-1_image_VH_2018-10-11.png
radius_4 ESIH sentinel-1_image_VH_2018-08-12.png
radius_4 ESIH sentinel-1_image_VH_2018-09-05.png
radius_4 ESIH sentinel-1_image_VH_2018-10-11.png
radius_4 ESIV sentinel-1_image_VH_2018-08-12.png
radius_4 ESIV sentinel-1_image_VH_2018-09-05.png
radius_4 ESIV sentinel-1_i

radius_6 SSI sentinel-1_image_VH_2019-03-04.png
radius_6 SSI sentinel-1_image_VH_2019-04-09.png
radius_6 SMPI sentinel-1_image_VH_2019-02-08.png
radius_6 SMPI sentinel-1_image_VH_2019-03-04.png
radius_6 SMPI sentinel-1_image_VH_2019-04-09.png
radius_6 MPI sentinel-1_image_VH_2019-02-08.png
radius_6 MPI sentinel-1_image_VH_2019-03-04.png
radius_6 MPI sentinel-1_image_VH_2019-04-09.png
radius_6 MPSSI sentinel-1_image_VH_2019-02-08.png
radius_6 MPSSI sentinel-1_image_VH_2019-03-04.png
radius_6 MPSSI sentinel-1_image_VH_2019-04-09.png
radius_6 ENL sentinel-1_image_VH_2019-02-08.png
radius_6 ENL sentinel-1_image_VH_2019-03-04.png
radius_6 ENL sentinel-1_image_VH_2019-04-09.png
radius_6 ESIH sentinel-1_image_VH_2019-02-08.png
radius_6 ESIH sentinel-1_image_VH_2019-03-04.png
radius_6 ESIH sentinel-1_image_VH_2019-04-09.png
radius_6 ESIV sentinel-1_image_VH_2019-02-08.png
radius_6 ESIV sentinel-1_image_VH_2019-03-04.png
radius_6 ESIV sentinel-1_image_VH_2019-04-09.png
